In [ ]:
### IF YOU ARE USING COLAB, UNCOMMENT AND RUN THIS BLOCK FIRST ###

# Mount google drive to allow access to your files
from google.colab import drive
drive.mount('/content/drive')
drive_folder = '/content/drive/MyDrive'
# Ajust this line to be the assignment1 folder in your google drive
notebook_folder = drive_folder + '/neuralEvolution'
%cd {notebook_folder}

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/neuralEvolution


In [ ]:
%cd ../../

/content/drive/MyDrive


In [ ]:
%cd ./neuralEvolution/datasets
!bash get_datasets.sh
%cd ../../

[Errno 2] No such file or directory: './neuralEvolution/datasets'
/content/drive/MyDrive/neuralEvolution
bash: get_datasets.sh: No such file or directory
/content/drive


In [ ]:
!pip install deap keras tensorflow matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 4.0 MB/s eta 0:00:00


In [ ]:
import random
import numpy as np
from deap import base, creator, tools, algorithms
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from keras.datasets import cifar10
from keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt


In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 55s 0us/step


1. We are first normalizing the inputs, since the input is a range from (0 to 255), we normalise them from 0 to 1, this helps during converging as they converge faster and even scaling enables the network to learn patterns more efficiently \\
2. We then do one-hot encoding since we will be using categorical crossentropy as our loss function, since the problem is a multi-class classification and this requires that all the classes be one-hot encoded

In [ ]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

num_classes = y_test.shape[1]
input_shape = x_train.shape[1:]


In [ ]:
def create_cnn(architecture):
    model = Sequential()
    added_flatten = False  # Track if a Flatten layer has been added

    for layer in architecture:
        if layer['type'] == 'conv' and not added_flatten:
            # Add Conv2D layer only if Flatten has not been added
            model.add(Conv2D(layer['filters'], (3, 3), activation=layer['activation'], input_shape=input_shape))
            model.add(MaxPooling2D(pool_size=(2, 2)))
        elif layer['type'] == 'dense':
            if not added_flatten:
                # Add Flatten layer before adding dense layers
                model.add(Flatten())
                added_flatten = True
            model.add(Dense(layer['units'], activation=layer['activation']))

    # Ensure the model has been flattened before adding the final dense layer
    if not added_flatten:
        model.add(Flatten())

    # Add the final output layer
    model.add(Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

    return model


def evaluate_cnn(architecture):
    model = create_cnn(architecture)
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=32, verbose=0)
    score = model.evaluate(x_test, y_test, verbose=0)
    return score[1]  # Return accuracy as fitness


In [ ]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

def generate_individual():
    layers = []
    num_layers = random.randint(2, 5)  # Random number of layers
    for _ in range(num_layers):
        layer_type = random.choice(['conv', 'dense'])
        if layer_type == 'conv':
            layers.append({
                'type': 'conv',
                'filters': random.choice([16, 32, 64]),
                'activation': random.choice(['relu', 'tanh'])
            })
        elif layer_type == 'dense':
            layers.append({
                'type': 'dense',
                'units': random.choice([64, 128, 256]),
                'activation': random.choice(['relu', 'tanh'])
            })
    return layers

toolbox = base.Toolbox()
toolbox.register("individual", tools.initIterate, creator.Individual, generate_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate_cnn)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)


/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [ ]:
def evolve_population():
    population = toolbox.population(n=10)  # Create a population of 10 individuals
    ngen = 5  # Number of generations
    cxpb = 0.5  # Crossover probability
    mutpb = 0.2  # Mutation probability

    for gen in range(ngen):
        print(f"-- Generation {gen} --")

        # Evaluate the entire population
        fitnesses = list(map(toolbox.evaluate, population))
        for ind, fit in zip(population, fitnesses):
            ind.fitness.values = (fit,)

        # Select the next generation individuals
        offspring = toolbox.select(population, len(population))

        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < cxpb:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < mutpb:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the new offspring
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = (fit,)

        # Replace population with offspring
        population[:] = offspring

    # Gather and return the best individual
    best_ind = tools.selBest(population, 1)[0]
    print(f"Best individual: {best_ind}")
    return best_ind


In [ ]:
best_architecture = evolve_population()
print("Best architecture found:", best_architecture)


-- Generation 0 --


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(None, 10, 2), output.shape=(None, 10)